In [this paper](https://arxiv.org/pdf/2306.04723), Tulchinskii et al. explore using persistent homology in detection of AI-generated texts.

Specifically, they use a feature called the "persistent homology dimension" of the text.
I hope to reproduce their work in this notebook, and apply it to my dataset.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
tqdm.pandas()

import kagglehub
# Download latest version
path = kagglehub.dataset_download("thedrcat/daigt-v2-train-dataset")
print("Path to dataset files:", path)
filename = 'train_v2_drcat_02.csv'

import spacy
from spacy.language import Language
from spacy.lang.en import English
from spacy.tokens import Doc, DocBin

import configparser

import ripser
import persim

pipeline_filepath = 'pipeline.spacy'
essays_filepath = 'parsed_essays.spacy'

Path to dataset files: /Users/kaveh/.cache/kagglehub/datasets/thedrcat/daigt-v2-train-dataset/versions/2


In [ ]:
#Reconstruct essay dataset from disk
doc_bin = DocBin().from_disk('parsed_essays.spacy')
data_filepath = os.path.join(path, filename)
essays = pd.read_csv(data_filepath)

nlp = spacy.load('en_core_web_md', exclude=['ner', 'parser']) # would like to use the saved pipeline, ideally. but this seems to work
essays['text_spacy'] = pd.Series(list(doc_bin.get_docs(nlp.vocab)))

essays.head()

,text,label,prompt_name,source,RDizzl3_seven,text_spacy
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False,"(Phones, Modern, humans, today, phone, phone, ..."
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False,"(essay, explain, drivers, able, use, electroni..."
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False,"(Driving, use, cellular, devices, Today, socie..."
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False,"(Phones, Driving, Drivers, able, use, phones, ..."
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False,"(Cell, Phone, Operation, Driving, ability, sta..."
